In [12]:
import os
os.chdir('c:\\Users\\abbaig\\Krish naik course\\heart-disease-prediction')



In [92]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [133]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

class DataTransformation:
    def __init__(self):
        pass

    def set_the_header(self, data_path):
        df = pd.read_csv(data_path)
        df = df.iloc[1:]
        df.columns = df.iloc[0]
        df = df.drop(df.index[0])
        df.reset_index(drop=True, inplace=True)
        return df

    def split_columns(self, df):
        df[['job', 'education']] = df['jobedu'].str.split(',', expand=True)
        df[['months', 'years']] = df['month'].str.split(",", expand=True)
        df.drop(['jobedu', 'month', 'years'], axis=1, inplace=True)
        return df

    def map_categorical(self, df):
        df['targeted'] = df['targeted'].map({'yes': 1, 'no': 0})
        df['default'] = df['default'].map({'yes': 1, 'no': 0})
        df['loan'] = df['loan'].map({'yes': 1, 'no': 0})
        df['housing'] = df['housing'].map({'yes': 1, 'no': 0})
        return df

    def convert_duration(self, df):
        df['duration'] = df['duration'].str.replace('sec', '').str.replace('min', '').astype(float) / 60
        return df

    def drop_columns(self, df):
        df.drop(['customerid', 'poutcome'], axis=1, inplace=True)
        return df

    def convert_data_types(self, df):
        df['day'] = df['day'].astype('int64')
        df = df.dropna(subset=['age'])
        df['age'] = df['age'].astype('int64')
        df['salary'] = df['salary'].astype(float)
        df['balance'] = df['balance'].astype(float)
        df['pdays'] = df['pdays'].astype('int64')
        df = df[df['pdays'] < 500]
        df['previous'] = df['previous'].astype('int64')
        df = df[df['previous'] < 100]
        return df

    def create_bins(self, df):
        bins_pdays = [-1, 0, 50, 100, 150, 200, 250, 300, 350, 400, 450, 500]
        labels_pdays = ['never-reached', '0-50', '50-100', '100-150', '150-200', '200-250', '250-300',
                        '300-350', '350-400', '400-450', '450-500']
        df['pdays-bin'] = pd.cut(df['pdays'], bins=bins_pdays, labels=labels_pdays, right=False)
        df.drop('pdays', axis=1, inplace=True)

        bins_previous = [0, 10, 30, 60, 90, 120]
        labels_previous = ['never answered', '10-30', '30-60', '60-90', '90+']
        df['previous-bin'] = pd.cut(df['previous'], bins=bins_previous, labels=labels_previous, right=False)
        df.drop('previous', axis=1, inplace=True)

        return df

    def one_hot_encode(self, df):
        list_of_categorical = ['marital', 'contact', 'day', 'campaign', 'job', 'education', 'months', 'pdays-bin',
                                'previous-bin']
        dummy = pd.get_dummies(df[list_of_categorical], drop_first=True)
        dummy = dummy.astype(int)
        df.drop(list_of_categorical, axis=1, inplace=True)
        df_one_hot_encoded = pd.concat([df, dummy], axis=1)
        
        return df_one_hot_encoded

    def scale_numerical(self, df):
        list_of_numerical = ['age', 'salary', 'balance', 'duration']
        scaler = MinMaxScaler()
        df_scaled = pd.DataFrame(scaler.fit_transform(df[list_of_numerical]),
                                 columns=[f'{col}_scaled' for col in list_of_numerical])
        df.drop(list_of_numerical, axis=1, inplace=True)
        df = pd.concat([df, df_scaled], axis=1)
        return df


In [101]:
# class DataTransformation:
#     def __init__(self):
#         pass
#     def set_the_header(self,data_path):
#         df = pd.read_csv(data_path)
#         df=df.iloc[1:]
#         df.columns=df.iloc[0]
#         df = df.drop(df.index[0])
#         df.reset_index(drop=True)
#         return df

#     def feature_engineering(self,df):
        
#         df[['job','education']]=df['jobedu'].str.split(',',expand=True)
#         df[['months','years']]=df.month.str.split(",",expand=True)
#         df.drop(['jobedu','month','years'],axis=1,inplace=True)
#         df.targeted=df.targeted.map({'yes':1,'no':0})
#         df.default=df.default.map({'yes':1,'no':0})
#         df.loan=df.loan.map({'yes':1,'no':0})
#         df.housing=df.housing.map({'yes':1,'no':0})
#         seconds_mask=df['duration'].astype(str).str.endswith('sec')
#         df.loc[seconds_mask,'duration']=df.loc[seconds_mask,'duration'].str.replace('sec', '').astype(float) / 60
#         minutes_mask=df['duration'].astype(str).str.endswith('min')
#         df.loc[minutes_mask,'duration']=df.loc[minutes_mask,'duration'].str.replace('min','').astype(float)
#         df.duration=df.duration.astype(float)
#         df.drop(['customerid','poutcome'],axis=1,inplace=True)
#         df.day=df.day.astype('int64')
#         df = df.dropna(subset=['age'])
#         df.age=df.age.astype('int64')
#         df.salary=df.salary.astype(float)
#         df.balance=df.balance.astype(float)
#         df.pdays=df.pdays.astype('int64')
#         df=df[df['pdays']<500] 
#         bins_pdays=[-1,0,50,100,150,200,250,300,350,400,450,500]
#         labels_pdays=['never-reached','0-50','50-100','100-150','150-200','200-250','250-300','300-350','350-400','400-450','450-500']
#         df['pdays-bin']=pd.cut(df['pdays'],bins=bins_pdays,labels=labels_pdays,right=False)
#         df.drop('pdays',axis=1,inplace=True)
#         df.previous=df.previous.astype('int64')
#         df= df[df['previous']<100]
#         bins_previous=[0,10,30,60,90,120]
#         labels_previous= ['never answered','10-30','30-60','60-90','90+']
#         df['previous-bin']= pd.cut(df['previous'],bins=bins_previous,labels=labels_previous,right=False)
#         df.drop(['previous'],axis=1,inplace=True)
#         list_of_categorical =['marital','contact','day','campaign','job','education','months','pdays-bin','previous-bin']
#         list_of_numerical =['age','salary','balance','duration']
#         dummy= pd.get_dummies(df[list_of_categorical],drop_first=True)
#         dummy=dummy.astype(int)
#         df_one_hot_encoded= pd.concat([df,dummy],axis=1)
#         scaler = MinMaxScaler()
#         df_scaled = pd.DataFrame(scaler.fit_transform(df_one_hot_encoded[list_of_numerical]),columns=['age_scaled','salary_scaled','balance_scaled','duration_scaled'])
#         df = pd.concat([df_one_hot_encoded,df_scaled],axis=1)
#         df.drop(list_of_numerical,axis=1,inplace=True)
#         df.drop(list_of_categorical,axis=1,inplace=True)
#         return df

In [134]:
# from DataTransformation import DataTransformation  # Assuming the class is in a file named DataTransformation.py

# Create an instance of the DataTransformation class
data_transformer = DataTransformation()

# Set the header and perform feature engineering
df = data_transformer.set_the_header('artifacts/data_ingestion/bank_marketing_updated_v1.csv')
df = data_transformer.split_columns(df)
df = data_transformer.map_categorical(df)
df = data_transformer.convert_duration(df)
df = data_transformer.drop_columns(df)
df = data_transformer.convert_data_types(df)
df = data_transformer.create_bins(df)
df = data_transformer.one_hot_encode(df)
df = data_transformer.scale_numerical(df)

# Print the head of the DataFrame
df.head()


C:\Users\abbaig\AppData\Local\Temp\ipykernel_22944\2922500169.py:9: DtypeWarning: Columns (0,1,2,3,11,14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_path)
C:\Users\abbaig\AppData\Local\Temp\ipykernel_22944\2922500169.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['age'] = df['age'].astype('int64')
C:\Users\abbaig\AppData\Local\Temp\ipykernel_22944\2922500169.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['salary'] = df['salary'].

,targeted,default,housing,loan,response,day,marital_married,marital_single,contact_telephone,contact_unknown,...,pdays-bin_400-450,pdays-bin_450-500,previous-bin_10-30,previous-bin_30-60,previous-bin_60-90,previous-bin_90+,age_scaled,salary_scaled,balance_scaled,duration_scaled
0,1.0,0.0,1.0,0.0,no,5.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.519481,0.833333,0.092259,0.067251
1,1.0,0.0,1.0,0.0,no,5.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.337662,0.500000,0.073067,0.038907
2,1.0,0.0,1.0,1.0,no,5.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.194805,1.000000,0.072822,0.019583
3,0.0,0.0,1.0,0.0,no,5.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.376623,0.166667,0.086476,0.023705
4,0.0,0.0,0.0,0.0,no,5.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.194805,0.000000,0.072812,0.051018


In [140]:
df.dropna(inplace=True)
df.head()

,targeted,default,housing,loan,response,day,marital_married,marital_single,contact_telephone,contact_unknown,...,pdays-bin_400-450,pdays-bin_450-500,previous-bin_10-30,previous-bin_30-60,previous-bin_60-90,previous-bin_90+,age_scaled,salary_scaled,balance_scaled,duration_scaled
0,1.0,0.0,1.0,0.0,no,5.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.519481,0.833333,0.092259,0.067251
1,1.0,0.0,1.0,0.0,no,5.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.337662,0.500000,0.073067,0.038907
2,1.0,0.0,1.0,1.0,no,5.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.194805,1.000000,0.072822,0.019583
3,0.0,0.0,1.0,0.0,no,5.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.376623,0.166667,0.086476,0.023705
4,0.0,0.0,0.0,0.0,no,5.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.194805,0.000000,0.072812,0.051018


In [103]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path

In [104]:
#update the configuration manager
from heart_disease_project.constants import * 
from heart_disease_project.utils.common import read_yaml,create_directories

class ConfigurationManager:
    def __init__(self, config_file_path=CONFIG_FILE_PATH, params_file_path=PARAMS_FILE_PATH, schema_file_path=SCHEMA_FILE_PATH):
        self.config=read_yaml(config_file_path)
        self.params=read_yaml(params_file_path)
        self.schema=read_yaml(schema_file_path)
        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self)-> DataTransformationConfig:
        config=self.config.data_transformation

        create_directories([config.root_dir])
        data_transformation_config= DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path
        )
        return data_transformation_config




In [142]:
import os
from sklearn.model_selection import train_test_split
from heart_disease_project import logger
import pandas as pd

class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.df = None  # Initialize df attribute

    def set_the_header(self, data_path):
        self.df = pd.read_csv(data_path)
        self.df = self.df.iloc[1:]
        self.df.columns = self.df.iloc[0]
        self.df = self.df.drop(self.df.index[0])
        self.df.reset_index(drop=True, inplace=True)
        return self.df

    def split_columns(self):
        self.df[['job', 'education']] = self.df['jobedu'].str.split(',', expand=True)
        self.df[['months', 'years']] = self.df['month'].str.split(",", expand=True)
        self.df.drop(['jobedu', 'month', 'years'], axis=1, inplace=True)
        return self.df

    def map_categorical(self):
        self.df['targeted'] = self.df['targeted'].map({'yes': 1, 'no': 0})
        self.df['default'] = self.df['default'].map({'yes': 1, 'no': 0})
        self.df['loan'] = self.df['loan'].map({'yes': 1, 'no': 0})
        self.df['housing'] = self.df['housing'].map({'yes': 1, 'no': 0})
        self.df['response']=self.df['response'].map({'yes':1,'no':0})
        return self.df

    def convert_duration(self):
        self.df['duration'] = self.df['duration'].str.replace('sec', '').str.replace('min', '').astype(float) / 60
        return self.df

    def drop_columns(self):
        self.df.drop(['customerid', 'poutcome'], axis=1, inplace=True)
        return self.df

    def convert_data_types(self):
        self.df['day'] = self.df['day'].astype('int64')
        self.df = self.df.dropna(subset=['age'])
        self.df['age'] = self.df['age'].astype('int64')
        self.df['salary'] = self.df['salary'].astype(float)
        self.df['balance'] = self.df['balance'].astype(float)
        self.df['pdays'] = self.df['pdays'].astype('int64')
        self.df = self.df[self.df['pdays'] < 500]
        self.df['previous'] = self.df['previous'].astype('int64')
        self.df = self.df[self.df['previous'] < 100]
        return self.df

    def create_bins(self):
        bins_pdays = [-1, 0, 50, 100, 150, 200, 250, 300, 350, 400, 450, 500]
        labels_pdays = ['never-reached', '0-50', '50-100', '100-150', '150-200', '200-250', '250-300',
                        '300-350', '350-400', '400-450', '450-500']
        self.df['pdays-bin'] = pd.cut(self.df['pdays'], bins=bins_pdays, labels=labels_pdays, right=False)
        self.df.drop('pdays', axis=1, inplace=True)

        bins_previous = [0, 10, 30, 60, 90, 120]
        labels_previous = ['never answered', '10-30', '30-60', '60-90', '90+']
        self.df['previous-bin'] = pd.cut(self.df['previous'], bins=bins_previous, labels=labels_previous, right=False)
        self.df.drop('previous', axis=1, inplace=True)

        return self.df

    def one_hot_encode(self):
        list_of_categorical = ['marital', 'contact', 'day', 'campaign', 'job', 'education', 'months', 'pdays-bin',
                                'previous-bin']
        dummy = pd.get_dummies(self.df[list_of_categorical], drop_first=True)
        dummy = dummy.astype(int)
        self.df.drop(list_of_categorical, axis=1, inplace=True)
        self.df = pd.concat([self.df, dummy], axis=1)
        return self.df

    def scale_numerical(self):
        list_of_numerical = ['age', 'salary', 'balance', 'duration']
        scaler = MinMaxScaler()
        df_scaled = pd.DataFrame(scaler.fit_transform(self.df[list_of_numerical]),
                                 columns=[f'{col}_scaled' for col in list_of_numerical])
        self.df.drop(list_of_numerical, axis=1, inplace=True)
        self.df = pd.concat([self.df, df_scaled], axis=1)
        print(self.df.shape)
        return self.df
    def drop_na(self):
        print(self.df.isnull().sum())
        self.df.dropna(inplace=True)
        print(self.df.shape)

    def convert_everything_to_float(self):
        

    
    def train_test_split(self):
        train, test = train_test_split(self.df, test_size=0.2, random_state=42)
        train.to_csv(os.path.join(self.config.root_dir, 'train.csv'), index=False)
        test.to_csv(os.path.join(self.config.root_dir, 'test.csv'), index=False)

        logger.info('Data split into train and test')
        logger.info(f'Train shape: {train.shape}')
        logger.info(f'Test shape: {test.shape}')
        return train, test


IndentationError: expected an indented block (1618333326.py, line 94)

In [ ]:
try:
    config = ConfigurationManager()
    data_transformation_config= config.get_data_transformation_config()
    data_transformation= DataTransformation(config=data_transformation_config)
    
    data_transformation.set_the_header(data_transformation_config.data_path)
    data_transformation.split_columns()
    data_transformation.map_categorical()
    data_transformation.convert_duration()
    data_transformation.drop_columns()
    data_transformation.convert_data_types()
    data_transformation.create_bins()
    data_transformation.one_hot_encode()
    data_transformation.scale_numerical()
    data_transformation.drop_na()
    data_transformation.train_test_split()
    
except Exception as e:
    raise e

[2024-03-27 18:54:00,035: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-03-27 18:54:00,035: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-27 18:54:00,051: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-03-27 18:54:00,051: INFO: common: created directory at: artifacts]
[2024-03-27 18:54:00,060: INFO: common: created directory at: artifacts/data_transformation]


TypeError: __init__() got an unexpected keyword argument 'config'